In [1]:
import pandas as pd 
AWM = pd.read_csv('/Users/5036365/Documents/Dravidian Lang Tech Data/Abusive Towards Women/AWM_train.csv')
len(AWM)

2933

In [2]:
AWM_val = pd.read_csv('/Users/5036365/Documents/Dravidian Lang Tech Data/Abusive Towards Women/AWM_dev.csv')
len(AWM_val)

629

In [3]:
import numpy as np
label = []
for i in range(0,len(AWM)):
    if AWM['Class'][i] =='Abusive':
        label.append(1)
    else:
        label.append(0)
AWM['label']=np.array(label)

In [4]:
label_val = []
for i in range(0,len(AWM_val)):
    if AWM_val['Class'][i] =='Abusive':
        label_val.append(1)
    else:
        label_val.append(0)
AWM_val['label']=np.array(label_val)

In [5]:
AWM = AWM.drop_duplicates(subset=['Text'])
AWM = AWM.reset_index()

In [12]:
AWM['label']=np.array(label)

In [7]:
from transformers import AutoTokenizer
 
import torch
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import os

# Split the data into train and test sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(AWM['Text'], AWM['label'], test_size=0.2)
 

train_texts, train_labels,test_texts, test_labels = AWM['Text'], AWM['label'],AWM_val['Text'], AWM_val['label']
# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
 
# Tokenize dataset
def preprocess_function(examples):
    return tokenizer(examples.to_list(), truncation=True, padding="max_length", max_length=128)
 
train_encodings = preprocess_function(train_texts)
test_encodings = preprocess_function(test_texts)

/Users/5036365/Documents/text-recommendation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
# Convert to Hugging Face Dataset format, and include 'labels' in the Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels  # Ensure labels are included
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels  # Ensure labels are included
})


In [9]:
from transformers import AutoModelForSequenceClassification
 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=5,              # Number of epochs
    learning_rate=5e-6,
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/1470 [00:00<?, ?it/s]

{'loss': 0.53, 'grad_norm': 25.739871978759766, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.03}
{'loss': 0.5916, 'grad_norm': 26.05185890197754, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.07}
{'loss': 0.5843, 'grad_norm': 19.190223693847656, 'learning_rate': 3.0000000000000004e-07, 'epoch': 0.1}
{'loss': 0.4778, 'grad_norm': 12.123152732849121, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.14}
{'loss': 0.3921, 'grad_norm': 16.33024024963379, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.17}
{'loss': 0.5107, 'grad_norm': 13.64146614074707, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.2}
{'loss': 0.5684, 'grad_norm': 28.327049255371094, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.24}
{'loss': 0.4881, 'grad_norm': 49.317325592041016, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.27}
{'loss': 0.4198, 'grad_norm': 8.839217185974121, 'learning_rate': 9.000000000000001e-07, 'epoch': 0.31}
{'loss': 0.3825, 'grad_norm': 18.498159408569336, 'learning

  0%|          | 0/74 [00:00<?, ?it/s]

{'eval_loss': 0.7901365756988525, 'eval_accuracy': 0.7103918228279387, 'eval_macro_f1': 0.7072302023189222, 'eval_weighted_f1': 0.7085259484291748, 'eval_runtime': 5.2228, 'eval_samples_per_second': 112.391, 'eval_steps_per_second': 14.169, 'epoch': 1.0}
{'loss': 0.5111, 'grad_norm': 45.90298080444336, 'learning_rate': 3e-06, 'epoch': 1.02}
{'loss': 0.4409, 'grad_norm': 18.731327056884766, 'learning_rate': 3.1000000000000004e-06, 'epoch': 1.05}
{'loss': 0.3417, 'grad_norm': 11.094863891601562, 'learning_rate': 3.2000000000000003e-06, 'epoch': 1.09}
{'loss': 0.4415, 'grad_norm': 77.6686782836914, 'learning_rate': 3.3000000000000006e-06, 'epoch': 1.12}
{'loss': 0.4759, 'grad_norm': 67.05368041992188, 'learning_rate': 3.4000000000000005e-06, 'epoch': 1.16}
{'loss': 0.453, 'grad_norm': 6.5480875968933105, 'learning_rate': 3.5e-06, 'epoch': 1.19}
{'loss': 0.4803, 'grad_norm': 30.33368492126465, 'learning_rate': 3.6000000000000003e-06, 'epoch': 1.22}
{'loss': 0.4648, 'grad_norm': 14.54588603

  0%|          | 0/74 [00:00<?, ?it/s]

...

{'eval_loss': 0.6838486194610596, 'eval_accuracy': 0.7103918228279387, 'eval_macro_f1': 0.7098655595096877, 'eval_weighted_f1': 0.7093392961914367, 'eval_runtime': 4.9199, 'eval_samples_per_second': 119.311, 'eval_steps_per_second': 15.041, 'epoch': 3.0}
{'loss': 0.3554, 'grad_norm': 19.752498626708984, 'learning_rate': 2.9896907216494846e-06, 'epoch': 3.03}
{'loss': 0.4137, 'grad_norm': 13.687132835388184, 'learning_rate': 2.9381443298969076e-06, 'epoch': 3.06}
{'loss': 0.1803, 'grad_norm': 6.8308844566345215, 'learning_rate': 2.8865979381443297e-06, 'epoch': 3.1}
{'loss': 0.5534, 'grad_norm': 42.67705535888672, 'learning_rate': 2.8350515463917527e-06, 'epoch': 3.13}
{'loss': 0.329, 'grad_norm': 18.903921127319336, 'learning_rate': 2.7835051546391757e-06, 'epoch': 3.16}
{'loss': 0.4537, 'grad_norm': 25.904035568237305, 'learning_rate': 2.731958762886598e-06, 'epoch': 3.2}
{'loss': 0.3251, 'grad_norm': 8.147713661193848, 'learning_rate': 2.680412371134021e-06, 'epoch': 3.23}
{'loss': 0

  0%|          | 0/74 [00:00<?, ?it/s]

{'eval_loss': 0.8261647820472717, 'eval_accuracy': 0.7257240204429302, 'eval_macro_f1': 0.7250057468087026, 'eval_weighted_f1': 0.7256043081705589, 'eval_runtime': 4.9347, 'eval_samples_per_second': 118.952, 'eval_steps_per_second': 14.996, 'epoch': 4.0}
{'loss': 0.3143, 'grad_norm': 9.614120483398438, 'learning_rate': 1.4948453608247423e-06, 'epoch': 4.01}
{'loss': 0.3, 'grad_norm': 74.0417709350586, 'learning_rate': 1.4432989690721649e-06, 'epoch': 4.05}
{'loss': 0.383, 'grad_norm': 61.64542770385742, 'learning_rate': 1.3917525773195878e-06, 'epoch': 4.08}
{'loss': 0.441, 'grad_norm': 21.30817985534668, 'learning_rate': 1.3402061855670104e-06, 'epoch': 4.12}
{'loss': 0.4028, 'grad_norm': 61.586124420166016, 'learning_rate': 1.288659793814433e-06, 'epoch': 4.15}
{'loss': 0.2382, 'grad_norm': 16.445781707763672, 'learning_rate': 1.2371134020618557e-06, 'epoch': 4.18}
{'loss': 0.2346, 'grad_norm': 82.2523422241211, 'learning_rate': 1.1855670103092783e-06, 'epoch': 4.22}
{'loss': 0.4241,

  0%|          | 0/74 [00:00<?, ?it/s]

{'eval_loss': 0.8420383334159851, 'eval_accuracy': 0.7189097103918228, 'eval_macro_f1': 0.7188574911830017, 'eval_weighted_f1': 0.7186943061554357, 'eval_runtime': 4.9449, 'eval_samples_per_second': 118.707, 'eval_steps_per_second': 14.965, 'epoch': 5.0}
{'train_runtime': 630.529, 'train_samples_per_second': 18.603, 'train_steps_per_second': 2.331, 'train_loss': 0.3941848554578768, 'epoch': 5.0}


In [10]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=7,              # Number of epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2569 [00:00<?, ?it/s]

{'loss': 0.6909, 'grad_norm': 1.379355549812317, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}
{'loss': 0.6966, 'grad_norm': 4.241412162780762, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.05}
{'loss': 0.6942, 'grad_norm': 5.781650066375732, 'learning_rate': 3e-06, 'epoch': 0.08}
{'loss': 0.6904, 'grad_norm': 4.858850002288818, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.11}
{'loss': 0.6932, 'grad_norm': 3.834796190261841, 'learning_rate': 5e-06, 'epoch': 0.14}
{'loss': 0.6895, 'grad_norm': 5.216373443603516, 'learning_rate': 6e-06, 'epoch': 0.16}
{'loss': 0.6946, 'grad_norm': 2.6693379878997803, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.19}
{'loss': 0.6891, 'grad_norm': 3.3522160053253174, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.22}
{'loss': 0.6861, 'grad_norm': 4.321838855743408, 'learning_rate': 9e-06, 'epoch': 0.25}
{'loss': 0.6945, 'grad_norm': 5.128048419952393, 'learning_rate': 1e-05, 'epoch': 0.27}
{'loss': 0.6845, 'grad_norm': 3.6345

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.7043213248252869, 'eval_accuracy': 0.48171701112877585, 'eval_macro_f1': 0.3251072961373391, 'eval_weighted_f1': 0.3132194299828736, 'eval_runtime': 5.3742, 'eval_samples_per_second': 117.04, 'eval_steps_per_second': 14.7, 'epoch': 1.0}
{'loss': 0.6889, 'grad_norm': 5.429834842681885, 'learning_rate': 3.7e-05, 'epoch': 1.01}
{'loss': 0.6816, 'grad_norm': 2.8305532932281494, 'learning_rate': 3.8e-05, 'epoch': 1.04}
{'loss': 0.7067, 'grad_norm': 1.5217540264129639, 'learning_rate': 3.9000000000000006e-05, 'epoch': 1.06}
{'loss': 0.7342, 'grad_norm': 3.6228187084198, 'learning_rate': 4e-05, 'epoch': 1.09}
{'loss': 0.6985, 'grad_norm': 1.6700778007507324, 'learning_rate': 4.1e-05, 'epoch': 1.12}
{'loss': 0.7025, 'grad_norm': 3.565525770187378, 'learning_rate': 4.2e-05, 'epoch': 1.14}
{'loss': 0.6971, 'grad_norm': 4.1544389724731445, 'learning_rate': 4.3e-05, 'epoch': 1.17}
{'loss': 0.6884, 'grad_norm': 1.777893304824829, 'learning_rate': 4.4000000000000006e-05, 'epoch': 1.2

...

{'eval_loss': 0.6934307217597961, 'eval_accuracy': 0.48171701112877585, 'eval_macro_f1': 0.3251072961373391, 'eval_weighted_f1': 0.3132194299828736, 'eval_runtime': 5.3543, 'eval_samples_per_second': 117.476, 'eval_steps_per_second': 14.755, 'epoch': 5.0}
{'loss': 0.6912, 'grad_norm': 3.0488932132720947, 'learning_rate': 1.7617206379893668e-05, 'epoch': 5.01}
{'loss': 0.6971, 'grad_norm': 1.274524211883545, 'learning_rate': 1.7375543740937653e-05, 'epoch': 5.04}
{'loss': 0.6991, 'grad_norm': 4.0959248542785645, 'learning_rate': 1.7133881101981632e-05, 'epoch': 5.07}
{'loss': 0.7011, 'grad_norm': 4.698570251464844, 'learning_rate': 1.6892218463025617e-05, 'epoch': 5.1}
{'loss': 0.6932, 'grad_norm': 4.494101047515869, 'learning_rate': 1.66505558240696e-05, 'epoch': 5.12}
{'loss': 0.7002, 'grad_norm': 5.745265007019043, 'learning_rate': 1.6408893185113584e-05, 'epoch': 5.15}
{'loss': 0.7042, 'grad_norm': 4.651430130004883, 'learning_rate': 1.6167230546157563e-05, 'epoch': 5.18}
{'loss': 0

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.6959190964698792, 'eval_accuracy': 0.48171701112877585, 'eval_macro_f1': 0.3251072961373391, 'eval_weighted_f1': 0.3132194299828736, 'eval_runtime': 5.4147, 'eval_samples_per_second': 116.165, 'eval_steps_per_second': 14.59, 'epoch': 6.0}
{'loss': 0.6774, 'grad_norm': 4.26944637298584, 'learning_rate': 8.675688738521025e-06, 'epoch': 6.02}
{'loss': 0.7018, 'grad_norm': 4.119719982147217, 'learning_rate': 8.434026099565008e-06, 'epoch': 6.05}
{'loss': 0.702, 'grad_norm': 5.241405487060547, 'learning_rate': 8.19236346060899e-06, 'epoch': 6.08}
{'loss': 0.6996, 'grad_norm': 4.0984063148498535, 'learning_rate': 7.950700821652974e-06, 'epoch': 6.1}
{'loss': 0.6926, 'grad_norm': 1.168696403503418, 'learning_rate': 7.709038182696956e-06, 'epoch': 6.13}
{'loss': 0.695, 'grad_norm': 3.9944839477539062, 'learning_rate': 7.4673755437409384e-06, 'epoch': 6.16}
{'loss': 0.6979, 'grad_norm': 1.585728645324707, 'learning_rate': 7.22571290478492e-06, 'epoch': 6.19}
{'loss': 0.6953, 'gr

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.6943172216415405, 'eval_accuracy': 0.48171701112877585, 'eval_macro_f1': 0.3251072961373391, 'eval_weighted_f1': 0.3132194299828736, 'eval_runtime': 5.3966, 'eval_samples_per_second': 116.554, 'eval_steps_per_second': 14.639, 'epoch': 7.0}
{'train_runtime': 1038.3216, 'train_samples_per_second': 19.773, 'train_steps_per_second': 2.474, 'train_loss': 0.6981403609573168, 'epoch': 7.0}


In [8]:
from transformers import AutoTokenizer
 
import torch
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import os
from transformers import BertTokenizer, BertForSequenceClassification

# Split the data into train and test sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(AWM['Text'], AWM['label'], test_size=0.2)
 

train_texts, train_labels,test_texts, test_labels = AWM['Text'], AWM['label'],AWM_val['Text'], AWM_val['label']
# Load tokenizer
model_name = "l3cube-pune/malayalam-bert"
tokenizer = BertTokenizer.from_pretrained(model_name)
 
# Tokenize dataset
def preprocess_function(examples):
    return tokenizer(examples.to_list(), truncation=True, padding="max_length", max_length=128)
 
train_encodings = preprocess_function(train_texts)
test_encodings = preprocess_function(test_texts)

In [9]:
# Convert to Hugging Face Dataset format, and include 'labels' in the Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels  # Ensure labels are included
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels  # Ensure labels are included
})


In [10]:
 
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/malayalam-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=10,              # Number of epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/3670 [00:00<?, ?it/s]

{'loss': 0.6938, 'grad_norm': 0.07485142350196838, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}
{'loss': 0.6933, 'grad_norm': 0.2428361475467682, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.05}
{'loss': 0.6936, 'grad_norm': 0.46446219086647034, 'learning_rate': 3e-06, 'epoch': 0.08}
{'loss': 0.6926, 'grad_norm': 0.46028611063957214, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.11}
{'loss': 0.6926, 'grad_norm': 0.45814818143844604, 'learning_rate': 5e-06, 'epoch': 0.14}
{'loss': 0.6933, 'grad_norm': 0.4715226888656616, 'learning_rate': 6e-06, 'epoch': 0.16}
{'loss': 0.6927, 'grad_norm': 0.24060001969337463, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.19}
{'loss': 0.693, 'grad_norm': 0.24683527648448944, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.22}
{'loss': 0.6927, 'grad_norm': 0.08412078022956848, 'learning_rate': 9e-06, 'epoch': 0.25}
{'loss': 0.6924, 'grad_norm': 0.4609825909137726, 'learning_rate': 1e-05, 'epoch': 0.27}
{'loss': 0.6904, 'grad

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.5802101492881775, 'eval_accuracy': 0.7297297297297297, 'eval_macro_f1': 0.7296470469255664, 'eval_weighted_f1': 0.7294741646986792, 'eval_runtime': 5.305, 'eval_samples_per_second': 118.568, 'eval_steps_per_second': 14.892, 'epoch': 1.0}
{'loss': 0.5475, 'grad_norm': 0.9835011959075928, 'learning_rate': 3.7e-05, 'epoch': 1.01}
{'loss': 0.5319, 'grad_norm': 11.820481300354004, 'learning_rate': 3.8e-05, 'epoch': 1.04}
{'loss': 0.5928, 'grad_norm': 1.3014060258865356, 'learning_rate': 3.9000000000000006e-05, 'epoch': 1.06}
{'loss': 0.6055, 'grad_norm': 16.292938232421875, 'learning_rate': 4e-05, 'epoch': 1.09}
{'loss': 0.5377, 'grad_norm': 11.673066139221191, 'learning_rate': 4.1e-05, 'epoch': 1.12}
{'loss': 0.608, 'grad_norm': 17.104536056518555, 'learning_rate': 4.2e-05, 'epoch': 1.14}
{'loss': 0.4897, 'grad_norm': 1.3011670112609863, 'learning_rate': 4.3e-05, 'epoch': 1.17}
{'loss': 0.5639, 'grad_norm': 1.5209540128707886, 'learning_rate': 4.4000000000000006e-05, 'epoch

  0%|          | 0/79 [00:00<?, ?it/s]

...

{'eval_loss': 1.0767096281051636, 'eval_accuracy': 0.7567567567567568, 'eval_macro_f1': 0.754212159521894, 'eval_weighted_f1': 0.7551266241531729, 'eval_runtime': 5.38, 'eval_samples_per_second': 116.914, 'eval_steps_per_second': 14.684, 'epoch': 8.0}
{'loss': 0.0195, 'grad_norm': 0.10474923253059387, 'learning_rate': 1.1514195583596216e-05, 'epoch': 8.01}
{'loss': 0.1207, 'grad_norm': 0.19335970282554626, 'learning_rate': 1.1356466876971609e-05, 'epoch': 8.04}
{'loss': 0.1284, 'grad_norm': 0.10387075692415237, 'learning_rate': 1.1198738170347003e-05, 'epoch': 8.07}
{'loss': 0.1031, 'grad_norm': 0.11810702085494995, 'learning_rate': 1.1041009463722398e-05, 'epoch': 8.09}
{'loss': 0.05, 'grad_norm': 0.10593099892139435, 'learning_rate': 1.0883280757097793e-05, 'epoch': 8.12}
{'loss': 0.1232, 'grad_norm': 0.10517764091491699, 'learning_rate': 1.0725552050473187e-05, 'epoch': 8.15}
{'loss': 0.0139, 'grad_norm': 74.88047790527344, 'learning_rate': 1.056782334384858e-05, 'epoch': 8.17}
{'lo

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 1.1089084148406982, 'eval_accuracy': 0.7519872813990461, 'eval_macro_f1': 0.7496325780771587, 'eval_weighted_f1': 0.7505204170345916, 'eval_runtime': 5.3779, 'eval_samples_per_second': 116.96, 'eval_steps_per_second': 14.69, 'epoch': 9.0}
{'loss': 0.1301, 'grad_norm': 1.3988842964172363, 'learning_rate': 5.678233438485804e-06, 'epoch': 9.02}
{'loss': 0.069, 'grad_norm': 0.09561020135879517, 'learning_rate': 5.520504731861199e-06, 'epoch': 9.05}
{'loss': 0.1247, 'grad_norm': 1.3640310764312744, 'learning_rate': 5.3627760252365935e-06, 'epoch': 9.07}
{'loss': 0.0673, 'grad_norm': 0.10314536094665527, 'learning_rate': 5.205047318611988e-06, 'epoch': 9.1}
{'loss': 0.0601, 'grad_norm': 0.09893060475587845, 'learning_rate': 5.047318611987382e-06, 'epoch': 9.13}
{'loss': 0.0106, 'grad_norm': 0.09745735675096512, 'learning_rate': 4.8895899053627764e-06, 'epoch': 9.16}
{'loss': 0.0104, 'grad_norm': 0.10037514567375183, 'learning_rate': 4.73186119873817e-06, 'epoch': 9.18}
{'loss':

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 1.0670872926712036, 'eval_accuracy': 0.7615262321144675, 'eval_macro_f1': 0.760299378074062, 'eval_weighted_f1': 0.7609264368058247, 'eval_runtime': 5.3238, 'eval_samples_per_second': 118.148, 'eval_steps_per_second': 14.839, 'epoch': 10.0}
{'train_runtime': 1369.2955, 'train_samples_per_second': 21.42, 'train_steps_per_second': 2.68, 'train_loss': 0.3127522908232355, 'epoch': 10.0}
